# Implementation of a Neural Network Classification Model
**Joey Shi**

In this notebook, we will describe the implementation details and theory behind neural network classification.
The topics covered will include the softmax loss function, computing gradients for multi-variable vector functions, and the gradient descent algorithm.

## Notation
1. $X$ is a $n \times d$ feature matrix of real numbers, such that each row $x_i\in\mathbb{R}^d$ is an example.
2. $y$ is an $n$-dimensional label vector, such that each $y_i\in\{1,2,\dots , k_L\}$ is a class label.
3. $h(z)$ is a non-linear function. We will call it an activation function.
4. $W^{(l)}$ is a $k_{l} \times k_{l-1}$ weight matrix.
5. $b^{(l)}$ is a $k_{l}$-dimensional bias vector.
6. $p(y_i \mid D, x_i)$ is the probability of predicting $y_i$ from $x_i$ and parameters $D$
7. $\delta_{a, b} = 1$ if $a = b$, $\delta_{a, b} = 0$ otherwise [Kronecker delta function].

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import gzip
import pickle

with gzip.open('data/mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding="latin1")

X, y = train_set
X_test, y_test = test_set
n, d = X.shape
print(f"n = {n}, d = {d}")

# Maximum Likelihood Estimate

We begin by defining our model's probability mass distribution:
\begin{align*}
    p(y_i \mid D, x_i) &= \frac{\exp\left(z_{i,y_i}^{(L)}\right)}{\sum_{c=1}^k \exp\left(z_{i,c}^{(L)}\right)} \tag{z_i^{(L)} \text{ is defined in section 4}}
\end{align*}
We choose this probability mass function for our classification problem because of its behaviour: $p(y_i \mid D, x_i)$ is larger if $z_{i,y_i}^{(L)} = \max_{c} z_{i,c}^{(L)}$ and small otherwise. For our model, we want to maximize this probability across all training example, so we maximize $p(y \mid D, X)$. Assuming that each example is i.i.d., we can equivalently minimize $-\log(p(y \mid D, X))$ to avoid underflow during computation.
\begin{align}
    -\log(p(y \mid D, X))
    &= -\log\left(\prod_{i=1}^n p(y_i = c \mid D, x_i)\right) \\
    &= \sum_{i=1}^n -\log(p(y_i \mid D, x_i)) \\
    &= \sum_{i=1}^n -z_{i, y_i}^{(L)} + \log\left(\sum_{c=1}^{k} \exp\left(z_{i, c}^{(L)}\right)\right) \tag{\text{Softmax Loss Function}}
\end{align}

In [ ]:
def softmax_loss():
    """Returns the softmax loss"""
    return 0

# Objective Function

Consider the following feed-forward neural network. For an input layer defined by $x_i$, our hidden layers are defined by the activation vectors $a_i^{(l)}$ and our output layer is defined by $z_i^{(L)}$. \vspace{1mm}

\noindent Note: We add hidden layers so our model can learn about latent factors in the data during training.
\begin{align*}
    a_i^{(0)} &= x_i & z_i^{(1)} &= W^{(1)}a_i^{(0)} + b^{(1)} \\
    a_i^{(1)} &= h\left(z^{(1)}\right) & z_i^{(2)} &= W^{(2)}a_i^{(1)} + b^{(2)} \\
    \vdots & & \vdots \\
    a_i^{(L-1)} &= h\left(z^{(L-1)}\right) & z_i^{(L)} &= W^{(L)}a_i^{(L-1)} + b^{(L)}
\end{align*}
After some weight initialization, given an example $x_i$, our model would predict $\displaystyle\hat{y}_i = \argmax_{c} z_{i,c}^{(L)}$. Using softmax loss, our objective function becomes
\begin{align*}
    f(W^{(1)},b^{(1)},\dots,W^{(L)},b^{(L)}) &= \sum_{i=1}^{n} \left[-z_{i, y_i}^{(L)} + \log\left(\sum_{c=1}^{k} \exp\left(z_{i, c}^{(L)}\right)\right)\right]
\end{align*}

# Gradients of the Objective Function
Let $A^{(l)}$ be a matrix such that each row is an activation vector $a_i^{(l)}$ and let $Z^{(l)}$ be defined similarly. Define a new matrix R by the recursion relation $R^{(l-1)} = R^{(l)}W^{(l)} \circ h\,'\left(Z^{(l-1)}\right)$ with a base case of $R^{(L)}$, where $r_{i, c}^{(L)} = p(y_i = c \mid D, x_i) - \delta_{y_i, c}$. Then the derivatives of the weights and biases of our network are given by \[\frac{\partial f}{\partial W^{(l)}} = \left(R^{(l)}\right)^TA^{(l-1)} \hspace{1cm} \frac{\partial f}{\partial b^{(l)}} = \sum_{i=1}^{n} r_i^{(l)} \]

\textbf{Gradient of $W^{(L)}$ and $b^{(L)}$}
\begin{align*}
    \frac{\partial f}{\partial w_{c,j}^{(L)}} &= \sum_{i=1}^{n} \left[-a_{i,j}^{(L-1)}\delta_{y_i, c} + \frac{1}{\sum_{c_0=1}^{k_l} \exp\left(z_{i,c_0}^{(L)}\right)}  \cdot \exp\left(z_{i,c}^{(L)}\right) \cdot a_{i,j}^{(L-1)}\right] &[\text{Chain Rule}] \\
    &= \sum_{i=1}^{n} [p(y_i = c \mid D, x_i) - \delta_{y_i,c}]a_{i,j}^{(L-1)} \\
    &= \sum_{i=1}^{n} r_{i,c}^{(L)}a_{i,j}^{(L-1)} \\
    \frac{\partial f}{\partial W^{(L)}} &= \left(R^{(L)}\right)^TA^{(L-1)} \\
    \frac{\partial f}{\partial b_{c}^{(L)}} &= \sum_{i=1}^{n} r_{i, c}^{(L)} \cdot 1 &[\text{Similar work as above}] \\
    \frac{\partial f}{\partial b^{(L)}} &= \sum_{i=1}^{n} r_i^{(L)}
\end{align*}

\newpage

\textbf{Gradient of $W^{(L-1)}$ and $b^{(L-1)}$}
\begin{align*}
    \frac{\partial f}{\partial w_{j, k}^{(L-1)}} &= \sum_{i=1}^{n} \sum_{c=1}^{k_L} \left[p(y_i = c \mid D, x_i) - \delta_{y_i, c}\right] \cdot w_{c,j}^{(L)}  h\,'\left(z_{j, k}^{(L-1)}\right) \cdot a_{j, k}^{(L-2)} &[\text{Chain Rule}]\\
    &= \sum_{i=1}^{n} \sum_{c=1}^{k_L} r_{i,c}^{(L)} \cdot w_{c,k}^{(L)}h\,'\left(z_{i, k}^{(L-1)}\right) \cdot a_{i, k}^{(L-2)} \\
    &= \sum_{i=1}^{n} \left[\left(W^{(L)}\right)^T r_i^{(L)} \circ h\,'\left(z_{i}^{(L-1)}\right)\right] \cdot a_i^{(L-2)} \\
    &= \sum_{i=1}^{n} r_{i}^{(L-1)} \cdot a_i^{(L-2)} \\
    &= \left(R^{(L-1)}\right)^T A^{(L-2)} \\
    \frac{\partial f}{\partial b_{k}^{(L-1)}} &= \sum_{i=1}^{n} r_{i, k}^{(L-1)} \cdot 1 &[\text{Similar work as above}] \\
    \frac{\partial f}{\partial b^{(L-1)}} &= \sum_{i=1}^{n} r_i^{(L-1)}
\end{align*}

\vspace{1cm}

\textbf{The following derivations are trivial and left as an easy exercise for the reader}

\vspace{1cm}

In [ ]:
def fun_obj():
    """Computes the objective function and gradient of the objective function"""
    raise NotImplemented

# Stochastic Gradient Descent Algorithm

Define $D$ to be a vector containing all the weights and biases in our model. Then $f(D)$ is our loss function. Given $E$ epoches, $B$ number of batches, and a learning rate $\alpha^t$, we proceed with minimizing $f(D)$ by stochastic gradient descent. Randomly initialize $D^0$. For each epoch, for each batch, sample $\lfloor n / B \rfloor$ training examples without replacement and compute $\nabla f(D^{t})$. Then,
\begin{align*}
    D^{t+1} &= D^{t} - \alpha^t \nabla f(D^{t})
\end{align*}
We will use the parameters $D^E$ to make predictions for the model.

In [ ]:
# Compute weights and biases

weights = []
biases = []

In [ ]:
def predict():
    pass

i = 50
image = 1 - np.reshape(X[i], (28, 28))
plt.imshow(image, cmap='gray')
print("Prediction: %d" %model.predict([X[i]]))
print("Actual: %d" %y[i])